## DB insert 
    - path 입력받아 DB insert : MySQL, AWS EC2  

In [6]:
# install

!pip install pymysql

  Using cached PyMySQL-1.0.2-py3-none-any.whl (43 kB)


In [2]:
import pandas as pd
import numpy as np

In [3]:
# df를 읽어올때 keep_default_na=False 옵션으로 NaN 값을 None으로 바꾸어 
# MySQL 데이터 추가 시 에러를 방지한다. 
# 파일 패스를 지정해 주고 사용하면 된다.

path = './synopsis_filter.csv'

In [4]:
# 데이터프레임 생성 # NaN 결측값을 확인한다.
data = pd.read_csv(path, keep_default_na=False)
data.isnull().sum()

serial         0
title          0
release        0
just_rating    0
imdb_rating    0
runtime        0
synopsis       0
director       0
actors         0
genre          0
poster_link    0
netflix        0
disney         0
wavve          0
watcha         0
dtype: int64

In [7]:
import pymysql

# DB에 connect
conn = pymysql.connect(host='3.35.111.73',
                       port=3306,
                       user='carbon',
                       password='1111',
                #        db='JI_MOVIS',
                       db='testdb',
                       charset='utf8mb4')

# 커서 생성
cursor = conn.cursor()

""" 쿼리문 작성 및 실행 """

# testdb 에 Base 테이블 생성
sql1 = '''CREATE TABLE testdb.test_1
        (       
                MV_NO int(13) NOT NULL PRIMARY KEY,
                TITLE varchar(4000),
                OPEN_DATE varchar(1000),
                JUST_RATING varchar(1000),
                IMDB_RATING varchar(1000),
                RUNTIME varchar(500),
                SYNOPSIS varchar(4000),
                DIRECTOR varchar(1000),
                ACTORS varchar(4000),
                GENRE varchar(4000),
                POSTER_LINK varchar(1000),
                NETFLIX int,
                DISNEY int,
                WAVVE int,
                WATCHA int
        ) '''
        
cursor.execute(sql1)

# user table 에 값 insert
sql2 = '''INSERT INTO test_1 (MV_NO, TITLE, OPEN_DATE, JUST_RATING, IMDB_RATING, 
                RUNTIME, SYNOPSIS, DIRECTOR, ACTORS, GENRE, POSTER_LINK, NETFLIX, DISNEY, WAVVE, WATCHA) 
                VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'''
                
for idx in range(len(data)):
    cursor.execute(sql2, tuple(data.values[idx]))  # data.values[idx] 는 배열이기에 튜플타입을 활용한다. 

conn.commit()
conn.close()

# with 방법을 고려하는 것도 가능하다. 오픈 클로즈 필요없이 with 구문이 끝나면 자동 종료된다. 
# 에러가 났을때 연결이 close() 되지 않는 것을 방지하는 차원에서 사용가능.

# with conn:
#     with conn.cursor() as cur:
#         cur.execute(sql1)
#         cur.execute(sql2, (value))
#         cur.execute(sql2, (value))
#         conn.commit()
### 커밋 후 with 구문으로 자동 종료.

OperationalError: (1118, 'Row size too large. The maximum row size for the used table type, not counting BLOBs, is 65535. This includes storage overhead, check the manual. You have to change some columns to TEXT or BLOBs')

- MySQL data를 확인해보도록 하자. ROW에 입력되는 데이터는 다음과 같다.

In [ ]:
data.columns            # 컬럼 확인.
data.values[1]          # ROW data 확인
tuple(data.values[0])   # tuple 형태로 변환된 값 확인.  

# 실제 사용 코드  _ 0401

In [ ]:
import pymysql

# DB에 connect
conn = pymysql.connect(host='3.35.111.73',
                       port=3306,
                       user='carbon',
                       password='1111',
                       db='JI_MOVIS',  
                       charset='utf8')

# 커서 생성
cursor = conn.cursor()

""" 쿼리문 작성 및 실행 """

# testdb 에 Base 테이블 생성
sql1 = '''CREATE TABLE JI_MOVIS.base
        (       
                MV_NO int(13) NOT NULL PRIMARY KEY,
                TITLE varchar(4000),
                OPEN_DATE varchar(1000),
                JUST_RATING varchar(1000),
                IMDB_RATING varchar(1000),
                RUNTIME varchar(500),
                SYNOPSIS text(65535),
                DIRECTOR varchar(1000),
                ACTORS text(100),
                GENRE varchar(4000),
                POSTER_LINK varchar(1000),
                NETFLIX int,
                DISNEY int,
                WAVVE int,
                WATCHA int
        ) '''
        
cursor.execute(sql1)

# user table 에 값 insert
sql2 = '''INSERT INTO base (MV_NO, TITLE, OPEN_DATE, JUST_RATING, IMDB_RATING, 
                RUNTIME, SYNOPSIS, DIRECTOR, ACTORS, GENRE, POSTER_LINK, NETFLIX, DISNEY, WAVVE, WATCHA) 
                VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'''
                
for idx in range(len(data)):
    cursor.execute(sql2, tuple(data.values[idx]))  # data.values[idx] 는 배열이기에 튜플타입을 활용한다. 

conn.commit()
conn.close()